# 일반 파서

In [2]:
from langchain.prompts import ChatPromptTemplate
from langchain.output_parsers import PydanticOutputParser
from langchain_openai import ChatOpenAI
from pydantic import BaseModel

# 출력 스키마 정의
class Item(BaseModel):
    title: str
    snippet: str

# 파서 생성
parser = PydanticOutputParser(pydantic_object=Item)

In [22]:

from typing import List
class Items(BaseModel):
    items: List[Item]  # 리스트 전체를 루트로 받음

# 파서 생성
parser = PydanticOutputParser(pydantic_object=Items)


In [26]:
# 프롬프트 템플릿
prompt = ChatPromptTemplate.from_template("""
아래 주제에 대해 하나의 제목과 그에 대한 한 줄 요약을 알려줘.
주제: {topic}
출력 형식: {{ "title": "...", "snippet": "..." }}
""")

# LLM 연결
llm = ChatOpenAI(model_name="gpt-4o-mini")

# 체인 연결
chain = prompt | llm | parser


In [27]:
import json
# 실행
results = chain.invoke({"topic": "인공지능의 미래"})
# 결과 출력
for item in results.items:
    print(f"제목: {item.title}\n요약: {item.snippet}\n")


OutputParserException: Failed to parse Items from completion {"title": "\uc778\uacf5\uc9c0\ub2a5\uc758 \ubbf8\ub798: \ud601\uc2e0\uacfc \ub3c4\uc804\uc758 \uae38", "snippet": "\uc778\uacf5\uc9c0\ub2a5\uc740 \uc6b0\ub9ac\uc758 \uc0b6\uc744 \ud601\uc2e0\uc801\uc73c\ub85c \ubcc0\ud654\uc2dc\ud0a4\ub294 \ub3d9\uc2dc\uc5d0 \uc724\ub9ac\uc801 \ub51c\ub808\ub9c8\uc640 \uc0ac\ud68c\uc801 \ub3c4\uc804 \uacfc\uc81c\ub97c \uc548\uaca8\uc8fc\uace0 \uc788\ub2e4."}. Got: 1 validation error for Items
items
  Field required [type=missing, input_value={'title': '인공지능... 안겨주고 있다.'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/missing
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE 

In [11]:
from dotenv import load_dotenv
from langchain_core.output_parsers import JsonOutputParser
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate


load_dotenv()

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

chain = llm | JsonOutputParser()


for chunk in chain.stream(
    "Output a JSON with 'answer' and 'source' for the question: What is the largest desert in the world?"
):
    print(chunk)

{}
{'answer': ''}
{'answer': 'The'}
{'answer': 'The largest'}
{'answer': 'The largest desert'}
{'answer': 'The largest desert in'}
{'answer': 'The largest desert in the'}
{'answer': 'The largest desert in the world'}
{'answer': 'The largest desert in the world is'}
{'answer': 'The largest desert in the world is the'}
{'answer': 'The largest desert in the world is the Antarctic'}
{'answer': 'The largest desert in the world is the Antarctic Desert'}
{'answer': 'The largest desert in the world is the Antarctic Desert.'}
{'answer': 'The largest desert in the world is the Antarctic Desert.', 'source': ''}
{'answer': 'The largest desert in the world is the Antarctic Desert.', 'source': 'National'}
{'answer': 'The largest desert in the world is the Antarctic Desert.', 'source': 'National Geographic'}


In [12]:
import langchain
from dotenv import load_dotenv
from langchain.output_parsers import (
    StructuredOutputParser,
    ResponseSchema,
)
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

load_dotenv()

response_schemas = [
    ResponseSchema(name="answer", description="사용자의 질문에 대한 답변"),
    ResponseSchema(
        name="source",
        description="사용자의 질문에 답하기 위해 사용된 출처를 표시해야 한다.",
    ),
]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

format_instructions = output_parser.get_format_instructions()
prompt = PromptTemplate(
    template="answer the users question as best as possible.\n{format_instructions}\n{question}",
    input_variables=["question"],
    partial_variables={"format_instructions": format_instructions},
)

llm = ChatOpenAI()

chain = prompt | llm | output_parser

question = "2002년 한일 월드컵에서 대한민국의 성적은?"
result = chain.invoke({"question": question})
print(result)

{'answer': '2002년 한일 월드컵에서 대한민국은 준결승에 진출하였습니다.', 'source': 'https://en.wikipedia.org/wiki/2002_FIFA_World_Cup'}


# StructuredOutputParser 

In [28]:

from langchain.prompts import ChatPromptTemplate
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain_openai import ChatOpenAI

# 스키마 정의
response_schemas = [
    ResponseSchema(name="title", description="주제에 대한 제목"),
    ResponseSchema(name="snippet", description="제목에 대한 한 줄 요약")
]

# 출력 파서 정의
parser = StructuredOutputParser.from_response_schemas(response_schemas)

# 포맷 지침
format_instructions = parser.get_format_instructions()

# 프롬프트 템플릿 정의
prompt = ChatPromptTemplate.from_template(f"""
다음 주제에 대해 총 3개의 항목을 만들어줘.
각 항목은 제목(title)과 그에 대한 한 줄 요약(snippet)으로 구성되어야 해.

주제: {{topic}}

아래 형식을 따르도록 해:
{format_instructions}

반드시 아래와 같이 출력해:
[
  {{
    "title": "제목1",
    "snippet": "요약1"
  }},
  {{
    "title": "제목2",
    "snippet": "요약2"
  }},
  {{
    "title": "제목3",
    "snippet": "요약3"
  }}
]
""")

# LLM 설정
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# 체인 연결
chain = prompt | llm | parser

# 실행
result = chain.invoke({"topic": "인공지능의 미래"})

# 출력
for item in result:
    print(f"제목: {item['title']}\n요약: {item['snippet']}\n")


KeyError: 'Input to ChatPromptTemplate is missing variables {\'\\n\\t"title"\', \'\\n    "title"\'}.  Expected: [\'\\n\\t"title"\', \'\\n    "title"\', \'topic\'] Received: [\'topic\']\nNote: if you intended {\n\t"title"} to be part of the string and not a variable, please escape it with double curly braces like: \'{{\n\t"title"}}\'.\nFor troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/INVALID_PROMPT_INPUT '

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain_openai import ChatOpenAI
from glom import glom

# 1. 출력 스키마 정의
response_schemas = [
    ResponseSchema(name="title", description="주제에 대한 제목"),
    ResponseSchema(name="snippet", description="제목에 대한 한 줄 요약")
]

# 2. 파서 생성
parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = parser.get_format_instructions()

# 3. 프롬프트 템플릿 (중괄호 완전 이스케이프 처리)
prompt = ChatPromptTemplate.from_template(f"""
다음 주제에 대해 총 3개의 항목을 만들어줘.
각 항목은 제목(title)과 그에 대한 한 줄 요약(snippet)으로 구성되어야 해.

주제: {{topic}}

출력 형식은 반드시 아래와 같이 작성해:
[
  {{{{
    "title": "제목1",
    "snippet": "요약1"
  }}}},
  {{{{
    "title": "제목2",
    "snippet": "요약2"
  }}}},
  {{{{
    "title": "제목3",
    "snippet": "요약3"
  }}}}
]

아래 형식 가이드를 참고해서 JSON만 반환해줘:
{format_instructions}
""")

# 4. LLM 설정
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# 5. 체인 연결
chain = prompt | llm | parser

# 6. 실행
result = chain.invoke({"topic": "인공지능의 미래"})

# 7. 출력
for item in result:
    print(f"제목: {item['title']}\n요약: {item['snippet']}\n")


KeyError: 'Input to ChatPromptTemplate is missing variables {\'\\n\\t"title"\'}.  Expected: [\'\\n\\t"title"\', \'topic\'] Received: [\'topic\']\nNote: if you intended {\n\t"title"} to be part of the string and not a variable, please escape it with double curly braces like: \'{{\n\t"title"}}\'.\nFor troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/INVALID_PROMPT_INPUT '